In [4]:
import pandas as pd
import numpy as np
import json
import ast
import os
from essentia.standard import MonoLoader

#### To do: Rodar o fluxo do zero novamente para gerar as labels e, na sequencia, remover musicas inválidas em outro script fazendo join com as musicas processadas 

In [5]:
from tqdm.notebook import tqdm

In [6]:
tqdm.pandas()

In [7]:

args = pd.Series({
    "root_dir":"/mnt/disks/data/",
    "dataset_path":"/mnt/disks/data/fma/fma_large",
    "embeddings":"music_style",
    "train_id": "hierarchical_mini_test",
    'sample_size':0.01
})


In [8]:



job_path = "/mnt/disks/data/fma/trains"


# In[15]:


train_path = os.path.join(job_path,args.train_id)


# In[16]:


base_path = os.path.join(args.root_dir,"fma")


# In[17]:


models_path = os.path.join(args.root_dir,"models")


metadata_path_fma = os.path.join(base_path,"fma_metadata")


# In[18]:


metadata_file = os.path.join(train_path,"metadata.json")


categories_labels_path = os.path.join(train_path,"labels.json")


In [9]:


def __load_json__(path):
    with open(path, 'r') as f:
        tmp = json.loads(f.read())

    return tmp




In [10]:

def create_dir(path):
    # checking if the directory demo_folder2 
    # exist or not.
    if not os.path.isdir(path):

        # if the demo_folder2 directory is 
        # not present then create it.
        os.makedirs(path)
    return True



In [11]:
create_dir(train_path)

True

## Load genres file. Contains relationships beetwen genres

In [12]:
genres_df = pd.read_csv(os.path.join(metadata_path_fma,'genres.csv'))


In [13]:
genres_df

,genre_id,#tracks,parent,title,top_level
0,1,8693,38,Avant-Garde,38
1,2,5271,0,International,2
2,3,1752,0,Blues,3
3,4,4126,0,Jazz,4
4,5,4106,0,Classical,5
...,...,...,...,...,...
158,1032,60,102,Turkish,2
159,1060,30,46,Tango,2
160,1156,26,130,Fado,2
161,1193,72,763,Christmas,38


In [14]:
genres_df[genres_df['genre_id'] == 495]

,genre_id,#tracks,parent,title,top_level
136,495,2061,15,Downtempo,15


In [37]:
# Cria um dicionário que associa o ID de cada música aos IDs de seus gêneros musicais
tracks_df = pd.read_csv(os.path.join(metadata_path_fma,'tracks_valid.csv'))

In [38]:
tracks_df.sample(20)

,track_id,track_title,valid_genre
31829,49242,omisaria de pasiones menores,"['32', '76']"
54560,82726,Обломали,"['25', '109']"
40494,61610,All For The Cause (Featuring Javier Starks),['21']
38259,58725,BWV 533: Prelude and Fugue in E Minor - 2. Fuga,['5']
42771,64682,Interlude,"['33', '38', '456']"
73347,113351,Magma / Dia do Sol,['19']
34436,53004,La guerre des fourmis,"['27', '32']"
82511,125385,Bring Me Home,"['10', '12', '1235']"
45336,67790,Junk Mail (Featuring BCMusic 1st),['21']
101538,151039,Pretty Boys,"['15', '183', '296']"


In [39]:
tracks_df.valid_genre.values

array(["['21']", "['21']", "['21']", ..., "['25']", "['25']",
       "['10', '12', '169']"], dtype=object)

In [40]:
tracks_df

,track_id,track_title,valid_genre
0,2,Food,['21']
1,3,Electric Ave,['21']
2,5,This World,['21']
3,10,Freeway,['10']
4,20,Spiritual Level,"['76', '103']"
...,...,...,...
104181,155316,The Auger,['25']
104182,155317,Let's Skin Ruby,['25']
104183,155318,My House Smells Like Kim Deal/Pulp,['25']
104184,155319,The Man With Two Mouths,['25']


In [41]:
## Get complete genre structure
def get_all_structure(estrutura,df_genres):
    ## Get structure from df_genres
    def get_all_structure_from_df(estrutura,df_genres,structure=[]):
        if estrutura == 0:
            return structure
        else:
            structure.append(estrutura)
            get_all_structure_from_df(df_genres[df_genres["genre_id"]==int(estrutura)].parent.values[0],df_genres,structure)
            return structure
    
    return get_all_structure_from_df(estrutura,df_genres,structure=[])
    

In [42]:
# tracks_df['valid_genre'] = tracks_df.track_genres.apply(lambda x: x.strip('][').split(', ') if x != '[]' else None)
tracks_df['valid_genre'] = tracks_df.valid_genre.apply(lambda x: ast.literal_eval(x))

In [44]:
tracks_df.loc[:,'last_genre_id'] = tracks_df.valid_genre.apply(lambda x:x[-1] if x != None else None)

In [45]:
tracks_df.sample(20)

,track_id,track_title,valid_genre,last_genre_id
47262,70960,cosmic lights,[495],495
99353,148039,It's Not Me!,"[10, 15, 21]",21
101848,151654,In This Galaxy (Toxic Chicken remix),[15],15
103998,155054,Broken Hill,"[15, 33, 38]",38
1468,1830,Do I Tingle? Up?,[12],12
37183,57239,Nulle Part Le Temps (with Severine Godissart),"[15, 26, 66]",66
38437,58903,BWV 1128: Wo Gott der Herr nicht bei uns hält,[5],5
92052,138067,Electric Lady,[10],10
23404,37001,kokw calling,"[15, 22]",22
78833,120457,Northlight,"[42, 468]",468


In [23]:
# tracks_df.dropna(inplace=True)

In [46]:
tracks_df

,track_id,track_title,valid_genre,last_genre_id
0,2,Food,[21],21
1,3,Electric Ave,[21],21
2,5,This World,[21],21
3,10,Freeway,[10],10
4,20,Spiritual Level,"[76, 103]",103
...,...,...,...,...
104181,155316,The Auger,[25],25
104182,155317,Let's Skin Ruby,[25],25
104183,155318,My House Smells Like Kim Deal/Pulp,[25],25
104184,155319,The Man With Two Mouths,[25],25


In [47]:
tracks_df.loc[:,'full_genre_id'] = tracks_df.last_genre_id.apply(lambda x: get_all_structure(x,genres_df))

In [48]:
tracks_df.full_genre_id.value_counts()

[38]             6508
[1235]           6043
[76, 10]         4126
[107, 1235]      3988
[103, 17]        3482
                 ... 
[493, 651, 9]       4
[189, 65, 20]       4
[173, 86, 2]        4
[444, 5]            2
[7, 20]             2
Name: full_genre_id, Length: 159, dtype: int64

In [49]:
def find_path(track_id,dataset_path):
    track_id = track_id.zfill(6)
    folder_id = track_id[0:3]
    file_path = os.path.join(dataset_path,folder_id,track_id+'.mp3')
    return file_path
    

In [50]:
tracks_df['file_path'] = tracks_df.track_id.apply(lambda x: find_path(str(x),args.dataset_path))

In [51]:
tracks_df.iloc[0].file_path

'/mnt/disks/data/fma/fma_large/000/000002.mp3'

In [52]:
tracks_df.columns

Index(['track_id', 'track_title', 'valid_genre', 'last_genre_id',
       'full_genre_id', 'file_path'],
      dtype='object')

In [53]:
tracks_df = tracks_df[['track_id','full_genre_id']]

In [ ]:
tracks_df

In [54]:
tracks_df['labels_size'] = tracks_df.full_genre_id.apply(lambda x: len(x))

<ipython-input-54-3052a08a907b>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tracks_df['labels_size'] = tracks_df.full_genre_id.apply(lambda x: len(x))


In [55]:
labels_size = tracks_df.labels_size.max()

In [56]:
labels_size

5

In [57]:
import pandas as pd
import os

### Parse of label to structure

In [58]:
### Function for parse label to sctructure of hierarhical scheme

def parse_label(label):
    # label = label.split('-')
    # preencher com 0 no caso de haver menos de 5 níveis
    labels = np.zeros(5,dtype=int)
    for i, label in enumerate(label):
        if i == 5:
            break
        # Aqui você pode fazer a conversão do label em um índice inteiro usando um dicionário ou outro método
        # Neste exemplo, estou apenas usando a posição da label na lista como índice
        labels[i] = label
    return labels

In [59]:
tracks_df

,track_id,full_genre_id,labels_size
0,2,[21],1
1,3,[21],1
2,5,[21],1
3,10,[10],1
4,20,"[103, 17]",2
...,...,...,...
104181,155316,"[25, 12]",2
104182,155317,"[25, 12]",2
104183,155318,"[25, 12]",2
104184,155319,"[25, 12]",2


In [60]:
tracks_df.full_genre_id.value_counts()

[38]             6508
[1235]           6043
[76, 10]         4126
[107, 1235]      3988
[103, 17]        3482
                 ... 
[493, 651, 9]       4
[189, 65, 20]       4
[173, 86, 2]        4
[444, 5]            2
[7, 20]             2
Name: full_genre_id, Length: 159, dtype: int64

In [61]:
tracks_df.columns

Index(['track_id', 'full_genre_id', 'labels_size'], dtype='object')

In [ ]:
tracks_df.loc[:,'labels'] = tracks_df.full_genre_id.apply(lambda x: parse_label(x))

In [ ]:
tracks_df.labels.value_counts()

In [178]:
def convert_label_to_string(x,level=2):
    return '-'.join([str(value) for value in x[:level]])

In [179]:
tracks_df['labels_1'] = tracks_df.labels.progress_apply(lambda x: str(x[:1][0]))
tracks_df['labels_2'] = tracks_df.labels.progress_apply(lambda x: convert_label_to_string(x,level=2))
tracks_df['labels_3'] = tracks_df.labels.progress_apply(lambda x: convert_label_to_string(x,level=3))
tracks_df['labels_4'] = tracks_df.labels.progress_apply(lambda x: convert_label_to_string(x,level=4))
tracks_df['labels_5'] = tracks_df.labels.progress_apply(lambda x: convert_label_to_string(x,level=5))

  0%|          | 0/104186 [00:00<?, ?it/s]

  0%|          | 0/104186 [00:00<?, ?it/s]

  0%|          | 0/104186 [00:00<?, ?it/s]

  0%|          | 0/104186 [00:00<?, ?it/s]

  0%|          | 0/104186 [00:00<?, ?it/s]

In [180]:
tracks_df.labels_2.unique()

array(['21-0', '10-0', '76-10', '45-12', '1-38', '17-0', '4-0', '25-12',
       '26-12', '30-38', '27-12', '15-0', '12-0', '31-12', '32-38',
       '89-25', '36-12', '3-0', '37-4', '41-38', '43-65', '79-2', '46-2',
       '11-14', '38-0', '2-0', '47-38', '33-17', '49-17', '250-38',
       '138-20', '53-45', '58-12', '22-38', '103-17', '64-25', '66-12',
       '42-15', '70-12', '71-25', '5-0', '98-12', '74-4', '85-12',
       '166-65', '188-20', '63-9', '9-0', '118-2', '111-25', '109-25',
       '97-4', '117-2', '92-2', '77-2', '13-0', '137-9', '102-2', '8-0',
       '185-15', '101-31', '224-38', '94-17', '20-0', '125-38', '180-17',
       '113-26', '7-20', '100-21', '906-4', '14-0', '538-18', '6-38',
       '107-1235', '18-1235', '286-15', '236-15', '90-53', '88-12',
       '297-15', '65-20', '337-15', '86-2', '763-16', '314-12', '167-31',
       '172-2', '179-4', '214-92', '19-14', '184-15', '181-15', '359-12',
       '361-109', '362-10', '183-15', '374-20', '81-92', '322-5',
       '

In [25]:
tracks_df.to_csv(os.path.join(train_path,"tracks.csv"),index=False)

In [14]:
tracks_df = pd.read_csv(os.path.join(train_path,"tracks.csv"))

In [21]:
labels_level_1 = tracks_df.labels_1.unique()
labels_level_2 = tracks_df.labels_2.unique()
labels_level_3 = tracks_df.labels_3.unique()
labels_level_4 = tracks_df.labels_4.unique()
labels_level_5 = tracks_df.labels_5.unique()

In [22]:
categories_df = pd.DataFrame(
    {'level5':labels_level_5,
    'level4':labels_level_4,
    'level3':labels_level_3,
    'level2':labels_level_2,
    'level1':labels_level_1})

In [23]:
def get_labels_name(x,genres_df):
    levels = 5
    full_name = []
    last_level = 0
    genre_root = ""
    for genre in x.split('-'):
        genre_df = genres_df[genres_df['genre_id'] == int(genre)]
        if genre_df.empty:
            genre_name = genre_root 
        else:
            genre_name = genre_df.title.values.tolist()[0]
            genre_root = genre_name
        
        full_name.append(genre_name)
    full_name = '>'.join(full_name)
        
    return full_name
    # return genres_df[genres_df['genre_id'] == int(x)].title.values.tolist()[0]

In [24]:
categories_df['level5_name'] = categories_df.level5.apply(lambda x: get_labels_name(x,genres_df))

In [25]:
categories_df['level5_name']

0                Hip-Hop>Hip-Hop>Hip-Hop>Hip-Hop>Hip-Hop
1                                    Pop>Pop>Pop>Pop>Pop
2                       Experimental Pop>Pop>Pop>Pop>Pop
3                          Loud-Rock>Rock>Rock>Rock>Rock
4      Avant-Garde>Experimental>Experimental>Experime...
                             ...                        
143     Symphony>Classical>Classical>Classical>Classical
144    Turkish>Middle East>International>Internationa...
145    Cumbia>Latin America>International>Internation...
146    N. Indian Traditional>Indian>International>Int...
147    Easy Listening: Vocal>Easy Listening>Easy List...
Name: level5_name, Length: 148, dtype: object

In [27]:
def __create_labels__(categories_df):
    data = {
        "label1": {},
        "label2": {},
        "label3": {},
        "label4": {},
        "label5": {},
        "label1_inverse": [],
        "label2_inverse": [],
        "label3_inverse": [],
        "label4_inverse": [],
        "label5_inverse": [],
        "label1_name": {},
        "label2_name": {},
        "label3_name": {},
        "label4_name": {},
        "label5_name": {},
    }

    for idx, cat in enumerate(set(categories_df.level1.values.tolist())):
        data['label1'][cat] = idx
        data['label1_inverse'].append(cat)
        data['label1_count'] = idx + 1

    for idx, cat in enumerate(set(categories_df.level2.values.tolist())):
        data['label2'][cat] = idx
        data['label2_inverse'].append(cat)
        data['label2_count'] = idx + 1
        
    for idx, cat in enumerate(set(categories_df.level3.values.tolist())):
        data['label3'][cat] = idx
        data['label3_inverse'].append(cat)
        data['label3_count'] = idx + 1

    for idx, cat in enumerate(set(categories_df.level4.values.tolist())):
        data['label4'][cat] = idx
        data['label4_inverse'].append(cat)
        data['label4_count'] = idx + 1
        
    for idx, cat in enumerate(set(categories_df.level5.values.tolist())):
        data['label5'][cat] = idx
        data['label5_inverse'].append(cat)
        data['label5_count'] = idx + 1
        
    for cat5,cat4,cat3,cat2,cat1,name5 in categories_df.values:
        
        name1 = '>'.join(name5.split('>')[:1])
        name2 = '>'.join(name5.split('>')[:2])
        name3 = '>'.join(name5.split('>')[:3])
        name4 = '>'.join(name5.split('>')[:4])
        
        
        data['label1_name'][cat1] = name1
        data['label2_name'][cat2] = name2
        data['label3_name'][cat3] = name3
        data['label4_name'][cat4] = name4
        data['label5_name'][cat5] = name5
        
    return data

In [28]:
with open(categories_labels_path, 'w+') as f:
    f.write(json.dumps(__create_labels__(categories_df)))